In [ ]:
#CODE FOR DAILY BASELOAD CSS OPTION VALUATION


import pandas as pd
import glob
import os
import numpy as np
import datetime as datetime

def listPatternFiles(searchDir, pattern):
    files = glob.glob(os.path.join(searchDir, pattern), recursive = True)
    return files

#Read the hourly power data:

today = pd.to_datetime('2022-01-12')
tYear = today.year
tMonth = today.month
tDay = today.day

strMonth = ''
strDay = ''

if len(str(tMonth)) < 2:
    strMonth = '0'+str(tMonth)
else:
    strMonth = str(tMonth)
    
if len(str(tDay)) < 2:
    strDay = '0'+str(tDay)
else:
    strDay = str(tDay)
    

pattern = 'Daily_'+str(tYear)+strMonth+strDay+'*HourlyReleveld.csv'
searchDir = 'I:/BU Portfolio Analytics/Market Analysis/Power/Models & Tools/Merit Order/PDP/Summary Outputs/'

HourlyData = pd.read_csv(listPatternFiles(searchDir, pattern)[0], sep = ";")

#Process the format and transform it to daily data:

HourlyData['VALUEDATETIME'] = pd.to_datetime(HourlyData['VALUEDATETIME'])
HourlyData = HourlyData[HourlyData['VALUEDATETIME'] >= pd.Timestamp(today)]
HourlyData['day'] = [str(i.year) + '-' + str(i.month) + '-' + str(i.day) for i in HourlyData['VALUEDATETIME']]
HourlyData = HourlyData.groupby('day').agg(np.mean)
HourlyData.index = pd.to_datetime(HourlyData.index)
HourlyData.sort_index(inplace = True)

#Calculate intrasimulation volatility for power:

Dailymean = HourlyData.T.mean()
for i in HourlyData.columns:
    HourlyData[i] = (HourlyData[i]/Dailymean.values)-1
    
PowerVolperday = HourlyData.T.std()
    
PowerVolperday = pd.DataFrame(PowerVolperday)
PowerVolperday.columns = ['RoughVol']
PowerVolperday['T'] = [i for i in range(1,len(PowerVolperday)+1)]
PowerVolperday['DailyVol'] = PowerVolperday['RoughVol']/np.sqrt(PowerVolperday['T'])
PowerVolperday['AnnualizedDailyVol'] = PowerVolperday['DailyVol']*np.sqrt(365)

PowerVolperday.to_csv('PowerVolperday.csv')

#Read gas and carbon daily data:

pattern = 'Daily_'+str(tYear)+strMonth+strDay+'*BASE CASE_Daily.csv'
    
gasAndCarbon = pd.read_csv(listPatternFiles(searchDir, pattern)[0], sep = ";")

#Process the format:

gasAndCarbon['VALUEDATETIME'] = pd.to_datetime(gasAndCarbon['VALUEDATETIME'])
gasAndCarbon = gasAndCarbon[gasAndCarbon['VALUEDATETIME'] >= pd.Timestamp(today)]

#Creating separated dataframes for Carbon and gas from the previous dataframe and setting their index:

carbonList = ['CARBON_Sim_'+str(i) for i in range(1,501)]
gasList = ['GAS_Sim_'+str(i) for i in range(1,501)]
carbonData = gasAndCarbon[carbonList]
gasData = gasAndCarbon[gasList]
carbonData.index, gasData.index = (gasAndCarbon['VALUEDATETIME'] for i in range(1,3))

#Creating E2 database:

E2Daily = pd.DataFrame(gasData.values/0.5305 - 0.3468*carbonData.values)
E2Daily.columns = ['E2_NL_Sim_'+str(i) for i in range(1,501)]
E2Daily.index = HourlyData.index
E2Daily['Year&Month'] = [str(i.year)+ '-' + str(i.month) for i in E2Daily.index]
E2Daily = E2Daily.iloc[:,:-1]

#Creating E2 Volatility database:

E2mean = E2Daily.T.mean()
for i in E2Daily.columns:
    E2Daily[i] = E2Daily[i]/E2mean-1

E2VolperDay = E2Daily.T.std()
E2VolperDay = pd.DataFrame(E2VolperDay)
E2VolperDay.columns = ['RoughVol']
E2VolperDay['T'] = [i for i in range(1,len(E2VolperDay)+1)]
E2VolperDay['DailyVol'] = E2VolperDay['RoughVol']/np.sqrt(E2VolperDay['T'])
E2VolperDay['AnnualizedDailyVol'] = E2VolperDay['DailyVol']*np.sqrt(365)

#Create CSV with E2 volatility:

E2VolperDay.to_csv('E2VolperDay.csv')